In [1]:
import numpy as np
from cobra import Model, Reaction, Metabolite
import utils

In [2]:
import importlib
importlib.reload(utils)

<module 'utils' from '/Users/gangl/Documents/GitHub/Halo-GEM/ComplementaryScripts/EnzymeConstrainedModelRecon/Scripts/utils.py'>

In [3]:
def report_model_rxns(model):
    for rxn in model.reactions: 
        print(rxn.id, rxn.name, rxn.gene_reaction_rule)
        print(rxn.reaction,'\n')

#### Use the toy model in Ben's paper

In [4]:
m1 = Metabolite('M1',compartment='c')
m2 = Metabolite('M2',compartment='c')
m3 = Metabolite('M3',compartment='c')
m4 = Metabolite('M4',compartment='c')

In [5]:
r1 = Reaction('R1',name='Reaction 1')
r1.add_metabolites({m1:1})
r1.lower_bound = 0
r1.upper_bound = 1000

r2 = Reaction('R2',name='Reaction 2')
r2.add_metabolites({m2:1})
r2.lower_bound = 0
r2.upper_bound = 1000

r3 = Reaction('R3',name='Reaction 3')
r3.add_metabolites({m1:-1,m2:-1,m3:1})
r3.gene_reaction_rule = 'E1'
r3.lower_bound = 0
r3.upper_bound = 1000

r4 = Reaction('R4',name='Reaction 4')
r4.add_metabolites({m3:-1,m4:1})
r4.gene_reaction_rule = 'E1 or E2'
r4.lower_bound = 0
r4.upper_bound = 1000

r5 = Reaction('R5',name='Reaction 5')
r5.add_metabolites({m2:-1,m4:1})
r5.gene_reaction_rule = 'E3 and E4'
r5.lower_bound = -1000
r5.upper_bound = 1000

r1 = Reaction('R6',name='Reaction 6')
r1.add_metabolites({m4:-1})
r1.lower_bound = 0
r1.upper_bound = 1000

In [6]:
model = Model()
model.add_reactions([r1,r2,r3,r4,r5])

Using license file /Users/gangl/gurobi.lic
Academic license - for non-commercial use only


In [7]:
report_model_rxns(model)

R6 Reaction 6 
M4 -->  

R2 Reaction 2 
 --> M2 

R3 Reaction 3 E1
M1 + M2 --> M3 

R4 Reaction 4 E1 or E2
M3 --> M4 

R5 Reaction 5 E3 and E4
M2 <=> M4 



#### Convert to irreversible model 

In [8]:
irrModel = utils.convertToIrrev(model)

In [9]:
report_model_rxns(irrModel)

R6 Reaction 6 
M4 -->  

R2 Reaction 2 
 --> M2 

R3 Reaction 3 E1
M1 + M2 --> M3 

R4 Reaction 4 E1 or E2
M3 --> M4 

R5 Reaction 5 E3 and E4
M2 --> M4 

R5_REV Reaction 5 (reversible) E3 and E4
M4 --> M2 



#### convert to ecmodel

In [10]:
kcats = {
     ('E1','R3'): 10,
     ('E1','R4'): 100,
     ('E2','R4'): 0.10,
     ('E3','R5'): 5,
     ('E3','R5_REV'): 12,
     ('E4','R5'): 15,
     ('E4','R5_REV'): 90,
       }
MWs = {
    'E1': 1000,
    'E2': 3000,
    'E3': 5000,
    'E4': 6000,
    'E5': 10000,
}

In [11]:
eModel = utils.convertToEnzymeModel(irrModel,kcats)

In [12]:
report_model_rxns(eModel)

R6 Reaction 6 
M4 -->  

R2 Reaction 2 
 --> M2 

R3 Reaction 3 E1
M1 + M2 + 0.1 prot_E1_c --> M3 

arm_R4 Reaction 4 (arm) 
M3 --> pmet_R4 

R4No1 Reaction 4 (No1) E1
pmet_R4 + 0.01 prot_E1_c --> M4 

R4No2 Reaction 4 (No2) E2
pmet_R4 + 10.0 prot_E2_c --> M4 

R5 Reaction 5 E3 and E4
M2 + 0.2 prot_E3_c + 0.2 prot_E4_c --> M4 

R5_REV Reaction 5 (reversible) E3 and E4
M4 + 0.08333333333333333 prot_E3_c + 0.08333333333333333 prot_E4_c --> M2 

prot_E1_exchange  E1
 --> prot_E1_c 

prot_E2_exchange  E2
 --> prot_E2_c 

prot_E3_exchange  E3
 --> prot_E3_c 

prot_E4_exchange  E4
 --> prot_E4_c 



#### add prot_pool  

In [13]:
ecModel = utils.constrainPool(eModel,MWs, ['E1','E2','E4'],10)

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpdc5_l3n0.lp
Reading time = 0.00 seconds
: 9 rows, 24 columns, 52 nonzeros


In [14]:
report_model_rxns(ecModel)

R6 Reaction 6 
M4 -->  

R2 Reaction 2 
 --> M2 

R3 Reaction 3 E1
M1 + M2 + 0.1 prot_E1_c --> M3 

arm_R4 Reaction 4 (arm) 
M3 --> pmet_R4 

R4No1 Reaction 4 (No1) E1
pmet_R4 + 0.01 prot_E1_c --> M4 

R4No2 Reaction 4 (No2) E2
pmet_R4 + 10.0 prot_E2_c --> M4 

R5 Reaction 5 E3 and E4
M2 + 0.2 prot_E3_c + 0.2 prot_E4_c --> M4 

R5_REV Reaction 5 (reversible) E3 and E4
M4 + 0.08333333333333333 prot_E3_c + 0.08333333333333333 prot_E4_c --> M2 

prot_E3_exchange  E3
 --> prot_E3_c 

draw_prot_E1 draw_prot_E1 
1000 prot_pool --> prot_E1_c 

draw_prot_E2 draw_prot_E2 
3000 prot_pool --> prot_E2_c 

draw_prot_E4 draw_prot_E4 
6000 prot_pool --> prot_E4_c 

prot_pool_exchange prot_pool_exchange 
 --> prot_pool 



In [15]:
ecModel.objective = 'R6'

In [16]:
ecModel.optimize()

,fluxes,reduced_costs
R6,0.008333,0.000000e+00
R2,0.008333,0.000000e+00
R3,0.000000,4.163336e-17
arm_R4,0.000000,0.000000e+00
R4No1,0.000000,0.000000e+00
...,...,...
prot_E3_exchange,0.001667,0.000000e+00
draw_prot_E1,0.000000,0.000000e+00
draw_prot_E2,0.000000,0.000000e+00
draw_prot_E4,0.001667,0.000000e+00
